# Comparing different Neural Networks

Each Network uses a vector of the eigen-eigen parts and the eigenvalues and computes the rest-eigen contribution

In [67]:
using LmaPredict, Flux, Statistics, Plots, Random, StatsBase, JLD2, PrettyTables

## Reading the data

In [68]:
const path_config = "/Users/lukasgeyer/Studium/Computational Sciences/Masterarbeit/Daten Simon/dat"
const path_plot = "../plots"

"../plots"

In [69]:
fname = readdir(path_config)[2:5001]
idx = sortperm( parse.(Int64, fname))
fname = fname[idx]
em_n = "VV"

cnfgarr = Vector{LMAConfig}(undef, 0)
for f in fname
    push!(cnfgarr, get_LMAConfig(joinpath(path_config, f), "g5-g5", em=em_n, bc=false))
end

## Splitting data in training and test sets

In [70]:
NCNFG = length(cnfgarr)
train_size = 500
test_size = NCNFG - train_size

TSRC = "24"
TVALS = length(cnfgarr[1].data["rr"][TSRC]) - 1
if em_n == "PA"
    EIGVALS = 32
else 
    EIGVALS = 64
end

eigvals_data_train = Array{Float32}(undef, EIGVALS, train_size)
rr_data_train = Array{Float32}(undef, TVALS, train_size)
ee_data_train = Array{Float32}(undef, TVALS, train_size)
re_data_train = Array{Float32}(undef, TVALS, train_size)

eigvals_data_test = Array{Float32}(undef, EIGVALS, test_size)
rr_data_test = Array{Float64}(undef, TVALS, test_size)
ee_data_test = Array{Float32}(undef, TVALS, test_size)
re_data_test = Array{Float64}(undef, TVALS, test_size)

for (k, dd) in enumerate(getfield.(cnfgarr, :data)[1:train_size])
    eigvals_data_train[:,k] = copy(cnfgarr[k].data["eigvals"][1:EIGVALS])
    rr_data_train[:,k] = getindex(getindex(dd, "rr"), TSRC)[2:end]
    ee_data_train[:,k] = getindex(getindex(dd, "ee"), TSRC)[2:end]
    re_data_train[:,k] = getindex(getindex(dd, "re"), TSRC)[2:end]
end
for (k, dd) in enumerate(getfield.(cnfgarr, :data)[train_size+1:NCNFG])
    eigvals_data_test[:,k] = copy(cnfgarr[k].data["eigvals"][1:EIGVALS])
    rr_data_test[:,k] = getindex(getindex(dd, "rr"), TSRC)[2:end]
    ee_data_test[:,k] = getindex(getindex(dd, "ee"), TSRC)[2:end]
    re_data_test[:,k] = getindex(getindex(dd, "re"), TSRC)[2:end]
end

## Defining training and test data

In [71]:
input_length = TVALS + EIGVALS
output_length = TVALS

input_shape_train = vcat(1 ./ eigvals_data_train, ee_data_train)
output_shape_train = re_data_train

input_shape_test = vcat(1 ./ eigvals_data_test, ee_data_test)
output_shape_test = re_data_test;

### Input

In [72]:
max_input_train = maximum(input_shape_train)
min_input_train = minimum(input_shape_train)

mean_input_train = mean(mean.([input_shape_train[:,i] for i in 1:train_size]))
std_input_train = std(mean.([input_shape_train[:,i] for i in 1:train_size]))

input_data_train_normalized = (input_shape_train .- max_input_train) ./ (max_input_train - min_input_train)
input_data_train_standardized = (input_shape_train .- mean_input_train) ./ std_input_train

input_data_test_normalized = (input_shape_test .- max_input_train) ./ (max_input_train - min_input_train)
input_data_test_standardized = (input_shape_test .- mean_input_train) ./ std_input_train;

### Output

In [73]:
max_output_train = maximum(output_shape_train)
min_output_train = minimum(output_shape_train)

mean_output_train = mean(mean.([output_shape_train[:,i] for i in 1:train_size]))
std_output_train = std(mean.([output_shape_train[:,i] for i in 1:train_size]))

output_data_train_normalized = (output_shape_train .- max_output_train) ./ (max_output_train - min_output_train)
output_data_train_standardized = (output_shape_train .- mean_output_train) ./ std_output_train;

## Describing different Neural Networks

In [12]:
activation_functions = [NNlib.tanh, NNlib.celu, NNlib.elu, NNlib.tanhshrink];

In [13]:
models = []

for activation_function in activation_functions
    push!(models,
        [
    Chain(
    Dense(input_length => 1, activation_function),
    Dense(1 => output_length, activation_function),
    ),
    Chain(
    Dense(input_length => 5, activation_function),
    Dense(5 => output_length, activation_function),
    ),
    Chain(
    Dense(input_length => 10, activation_function),
    Dense(10 => output_length, activation_function),
    ),
    Chain(
    Dense(input_length => 50, activation_function),
    Dense(50 => output_length, activation_function),
    ),
    Chain(
    Dense(input_length => 5, activation_function),
    Dense(5 => 1, activation_function),
    Dense(1 => output_length, activation_function)
    ),
    Chain(
    Dense(input_length => 10, activation_function),
    Dense(10 => 1, activation_function),
    Dense(1 => output_length, activation_function),
    ),
    Chain(
    Dense(input_length => 50, activation_function),
    Dense(50 => 1, activation_function),
    Dense(1 => output_length, activation_function),
    ),
    Chain(
    Dense(input_length => 10, activation_function),
    Dense(10 => 5, activation_function),
    Dense(5 => output_length, activation_function),
    ),
    Chain(
    Dense(input_length => 50, activation_function),
    Dense(50 => 5, activation_function),
    Dense(5 => output_length, activation_function),
    ),
    Chain(
    Dense(input_length => 50, activation_function),
    Dense(50 => 5, activation_function),
    Dense(5 => 1, activation_function),
    Dense(1 => output_length, activation_function),
    ),
    Chain(
    Dense(input_length => 50, activation_function),
    Dense(50 => 50, activation_function),
    Dense(50 => output_length, activation_function),
    ),
    Chain(
    Dense(input_length => 2*input_length, activation_function),
    Dense(2*input_length => 2*input_length, activation_function),
    Dense(2*input_length => output_length, activation_function),
    ),
    Chain(
    Dense(input_length => 2*input_length, activation_function),
    Dense(2*input_length => input_length, activation_function),
    Dense(input_length => output_length, activation_function),
    Dense(output_length => output_length, activation_function),
    ),
    Chain(
    Dense(input_length => 2*input_length, activation_function),
    Dense(2*input_length => 2*input_length, activation_function),
    Dense(2*input_length => input_length, activation_function),
    Dense(input_length => output_length, activation_function),
    ),
    Chain(
    Dense(input_length => 2*input_length, activation_function),
    Dense(2*input_length => 2*input_length, activation_function),
    Dense(2*input_length => 2*input_length, activation_function),
    Dense(2*input_length => output_length, activation_function),
    )
     ])
end

models = vcat(models...);

## Describing parameter regions

In [14]:
learning_rates = [0.1, 0.01, 0.001];

In [15]:
optimizers = []
push!(optimizers,[Flux.AdaGrad(), Flux.AdaDelta(), Flux.AMSGrad(), Flux.NAdam()])
for learning_rate in learning_rates
    push!(optimizers,
        [
            Flux.Adam(learning_rate),
            Flux.RAdam(learning_rate),
            Flux.AdaMax(learning_rate),
            Flux.AdamW(learning_rate),
            Flux.OAdam(learning_rate)
            ])
end
optimizers = vcat(optimizers...);

In [16]:
loss_functions = [Flux.mse, Flux.mae]

epochs = [1_000, 10_000, 50_000, 100_000, 200_000]

batch_sizes = [32, 64];

# Perfoming Grid-Search

In [ ]:
n_scores = length(models) * length(loss_functions) * length(epochs) * length(batch_sizes) * length(optimizers)

scores_table_header = ["Index", "|σ_p - σ_t|", "max(|μ_p - μ_t| / μ_t)"]
lookup_table_header = ["Index", "Model", "Optimizer", "Loss Function", "Epochs", "Batch Size"]

scores_mat = Matrix{Float64}(undef, n_scores, length(scores_table_header))
lookup_mat = Matrix{Any}(undef, n_scores, length(lookup_table_header))

index = 1

for model in models
    for optimizer in optimizers
        for loss_function in loss_functions
            for epoch in epochs
                for batch_size in batch_sizes
                    
                    loader = Flux.DataLoader(
                        (input_data_train_normalized, output_data_train_normalized),
                        batchsize=batch_size,
                        shuffle=true)

                    optim = Flux.setup(optimizer, model)

                    function training()
                        for e in 1:epoch
                            for (x,y) in loader
                                _, grads = Flux.withgradient(model) do m
                                    y_hat = m(x)
                                    loss_function(y_hat, y)
                                end
                                Flux.update!(optim, model, grads[1])
                            end
                        end
                    end

                    training()

                    out_of_sample_predictions = (model(input_data_test_normalized) .* (max_output_train - min_output_train)) .+ max_output_train
                    out_of_sample_error = [loss_function(out_of_sample_predictions[:,i], output_shape_test[:,i]) for i in 1:test_size]

                    mean_target = mean.([output_shape_test[i,:] for i in 1:TVALS])
                    σ_target = std.([output_shape_test[i,:] for i in 1:TVALS]) ./ sqrt(test_size - 1)
        
                    mean_predicted = mean.([out_of_sample_predictions[i,:] for i in 1:TVALS])
                    σ_predicted = std.([out_of_sample_predictions[i,:] for i in 1:TVALS]) ./ sqrt(test_size - 1)

                    mean_Δσ = mean(abs.(σ_target - σ_predicted))
                    max_Δμ = maximum(abs.((mean_target - mean_predicted) ./ mean_target))

                    scores_mat[index,:] = [index, mean_Δσ, max_Δμ]

                    lookup_mat[index,:] = [index, model, optimizer, loss_function, epoch, batch_size]

                    Flux.loadparams!(model,map(p -> p .= randn.(), Flux.params(model)))
                    index += 1
                end
            end
        end
    end
end

In [ ]:
output_dir = "../gridSearch/ee+ev/"

scores_file = output_dir * "scores.txt"
scores_file_mean = output_dir * "scores_meanSorted.txt"
lookup_file = output_dir * "lookup.txt"

mkpath(output_dir)
rm(scores_file, force=true)
rm(scores_file_mean, force=true)
rm(lookup_file, force=true)

open(scores_file, "a") do file
    pretty_table(file, scores_mat[sortperm(scores_mat[:, 2]), :], header=scores_table_header)
end

open(scores_file_mean, "a") do file
    pretty_table(file, scores_mat[sortperm(scores_mat[:, 3]), :], header=scores_table_header)
end

open(lookup_file, "a") do file
    pretty_table(file, lookup_mat, header=lookup_table_header)
end

# Test suite

In [ ]:
#index = 2967
index = 2609

optimizer = lookup_mat[index,3]
loss_function = lookup_mat[index,4]
epochs = lookup_mat[index, 5]
batch_size = lookup_mat[index, 6]

model = lookup_mat[index,2]

In [ ]:
Flux.loadparams!(model,map(p -> p .= randn.(), Flux.params(model)))

In [ ]:
loader = Flux.DataLoader((input_data_train_normalized, output_data_train_normalized), batchsize=batch_size, shuffle=true)
optim = Flux.setup(optimizer, model)

function training()
    losses = []
    for epoch in 1:epochs
        for (x, y) in loader
            loss, grads = Flux.withgradient(model) do m
                y_hat = m(x)
                loss_function(y_hat,y)
            end
            Flux.update!(optim, model, grads[1])
            push!(losses, loss) 
        end
    end
        
    return losses
end

losses = training()

out_of_sample_predictions = (model(input_data_test_normalized) .* (max_output_train - min_output_train)) .+ max_output_train
out_of_sample_error = [loss_function(out_of_sample_predictions[:,i], output_shape_test[:,i]) for i in 1:test_size];

In [ ]:
l = @layout [a b c; d e f; g h i]

c1 = rand([i for i in 1:500])
p1 = scatter(output_shape_test[:,c1], label="Actual")
scatter!(p1, out_of_sample_predictions[:,c1], label="Prediction", legend=:top)

c2 = rand([i for i in 1:500])
p2 = scatter(output_shape_test[:,c2], label="Actual")
scatter!(p2, out_of_sample_predictions[:,c2], label="Prediction", legend=:top)

c3 = rand([i for i in 1:500])
p3 = scatter(output_shape_test[:,c3], label="Actual")
scatter!(p3, out_of_sample_predictions[:,c3], label="Prediction", legend=:top, )

c4 = rand([i for i in 1:500])
p4 = scatter(output_shape_test[:,c4], label="Actual")
scatter!(p4, out_of_sample_predictions[:,c4], label="Prediction", legend=:top)

c5 = rand([i for i in 1:500])
p5 = scatter(output_shape_test[:,c5], label="Actual")
scatter!(p5, out_of_sample_predictions[:,c5], label="Prediction", legend=:top)

c6 = rand([i for i in 1:500])
p6 = scatter(output_shape_test[:,c6], label="Actual")
scatter!(p6, out_of_sample_predictions[:,c6], label="Prediction", legend=:top)

c7 = rand([i for i in 1:500])
p7 = scatter(output_shape_test[:,c7], label="Actual")
scatter!(p7, out_of_sample_predictions[:,c7], label="Prediction", legend=:top)

c8 = rand([i for i in 1:500])
p8 = scatter(output_shape_test[:,c8], label="Actual")
scatter!(p8, out_of_sample_predictions[:,c8], label="Prediction", legend=:top)

c9 = rand([i for i in 1:500])
p9 = scatter(output_shape_test[:,c9], label="Actual")
scatter!(p9, out_of_sample_predictions[:,c9], label="Prediction", legend=:top)

plot(p1, p2, p3, p4, p5, p6, p7, p8, p9, layout = l, size=(1200,1000), dpi=1000, markerstrokewidth = 0)
#savefig(joinpath(path_plot, "neural_network_test.pdf"))

In [ ]:
mean_target = mean.([output_shape_test[i,:] for i in 1:TVALS])
σ_target = std.([output_shape_test[i,:] for i in 1:TVALS]) ./ sqrt(test_size - 1)
        
mean_predicted = mean.([out_of_sample_predictions[i,:] for i in 1:TVALS])
σ_predicted = std.([out_of_sample_predictions[i,:] for i in 1:TVALS]) ./ sqrt(test_size - 1)

max_Δμ = maximum(abs.((mean_target - mean_predicted) ./ mean_target))

p = scatter(
        size=(1400,1000),
        dpi = 1000,
        thickness_scaling = 1.5,
)
    
scatter!(p,
    mean_target[7:41],
    yerr=σ_target[7:41],
    label="actual",
    legend=:bottom,
    linecolor=:blue,
    marker=:xcross,
    markersize = 5,
    markerstrokewidth = 0.3
)
scatter!(p,
    mean_predicted[7:41],
    yerr=σ_predicted[7:41],
    label="predicted",
    legend=:bottom,
    linecolor=:red,
    marker =:+,
    markersize = 5,
    markerstrokewidth = 0.3
)